In [ ]:
######CIRCLE TRAJECTORY########

#Simulation parameters
T_HORIZON = 20.0
DT=0.05
SIM_HORIZON = int(T_HORIZON/DT)

#control parameters
vMax = 1.5
vMin = 0.2
angleMax = np.pi * (1/7.2)

# trajectory initialization
x0 = np.array([0.0, 0.0, 0.0])

#hisotry
x_t = np.zeros((SIM_HORIZON, 3))
u_t = np.zeros((SIM_HORIZON, 2))


x_t[0]=x0
u=np.array([0.0, 0.0])

for i in range(0, SIM_HORIZON-1):

    u[0]=vMin
    u[1]=angleMax
    
    #propagate the system using the filtered control and save history
    x_t[i+1] = TVD_rk3(x_t[i], rc3D_dyn, u, DT)
    u_t[i] = u


# Extract x and y coordinates
x_coords = x_t[:, 0]
y_coords = x_t[:, 1]

# Create scatter plot
plt.scatter(x_coords, y_coords, c='blue', marker='o')
plt.title('Trajectory of RC Car')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.grid(True)
plt.show()

# print the maximum x and y values


In [ ]:
####Backup plot section as ipywidgets breaks sometimes####

idx_to_plot = 0

#def plot_func(idx_to_plot):
    
state_plot = x_t[idx_to_plot]
#print the state with 2 decimals
print('[x,y,th]')
print(np.around(state_plot, decimals=2))
#get the index of the closest v and th to the state_first_fltr
th_idx = np.argmin(np.abs(matlab_var_dict['th_coord'] - state_plot[2]))

f1, ax1 = plt.subplots()
data1 = data_lx
data1 = data1.transpose()
CP1 = ax1.contour(matlab_var_dict['x_coord'],matlab_var_dict['y_coord'],data1,0)
for c in CP1.collections:
    c.set_edgecolor("red")
    c.set_linewidth(1)
    c.set_label("")

data2 = data[:,:,th_idx] #slice of lx
data2 = data2.transpose()
CP2 = ax1.contour(matlab_var_dict['x_coord'],matlab_var_dict['y_coord'],data2,0)
for c in CP2.collections:
    c.set_edgecolor("blue")
    c.set_linewidth(1)
    c.set_label("")

#plot an arrow with angle theta
th=matlab_var_dict['th_coord']
th = th[th_idx]

SP1 = ax1.scatter(x_t[0:idx_to_plot,0], x_t[0:idx_to_plot,1],s=5,c=m_t[0:idx_to_plot], alpha=0.5)
SP2 = ax1.scatter(x_t[idx_to_plot,0], x_t[idx_to_plot,1],s=10, c=[[0.0,0.0,0.0]], alpha=1.0)
#plot the arrow
ax1.arrow(x_t[idx_to_plot,0], x_t[idx_to_plot,1], np.cos(th)*0.2, np.sin(th)*0.2, head_width=0.1, head_length=0.1, fc='k', ec='k')

#plot the hallucinations corresponding to idx_to_plot
for h in list_hallucinations_at_idx[idx_to_plot]:
    #remove zero rows
    h = h[~np.all(h == 0, axis=1)]
    ax1.plot(h[:,0], h[:,1], color='green', alpha=0.2)

#set x and y limits
ax1.set_xlim([0, 6])
ax1.set_ylim([0, 4])

#change aspect ratio to match the grid
ax1.set_aspect('equal')
plt.show()

#interact(plot_func, idx_to_plot=(0, i, 1))

In [5]:
#BRT plot comparison

import numpy as np
import matplotlib.pyplot as plt
import mat73
import time

import jax
import jax.numpy as jnp
from jax import random, jit
import matplotlib.pyplot as plt



# ------------------     MATLAB STUFF  ----------------------------------
def get_matlab_variables(mat_file_path):
    variables = mat73.loadmat(mat_file_path)
    #double gets converted to np array by default
    value_func_data = variables['Vx']
    lx_data = variables['lx'] 
    tau2 = variables['tau2']

    #Deriv is cell which gets converted into list of lists
    deriv_x_data = np.array(variables['Deriv'][0])
    deriv_x_data = deriv_x_data.squeeze()
    deriv_y_data = np.array(variables['Deriv'][1])
    deriv_y_data = deriv_y_data.squeeze()
    deriv_th_data = np.array(variables['Deriv'][2])
    deriv_th_data = deriv_th_data.squeeze()
    
    #uOpt is also cell which gets converted into list of lists
    uOpt_vel = np.array(variables['uOpt'][0])
    uOpt_vel = uOpt_vel.squeeze()
    uOpt_angle = np.array(variables['uOpt'][1])
    uOpt_angle = uOpt_angle.squeeze() 

    #g is struct whic gets converted into dic
    #vs is cell which give a list
    x_coord=np.array(variables['g']['vs'][0])
    y_coord=np.array(variables['g']['vs'][1])
    th_coord=np.array(variables['g']['vs'][2])
    x_coord = x_coord.squeeze()
    y_coord = y_coord.squeeze()
    th_coord = th_coord.squeeze()

    matlab_var_dict = dict( value_func_data=value_func_data,
                            lx_data=lx_data,
                            deriv_x_data=deriv_x_data,
                            deriv_y_data=deriv_y_data,
                            deriv_th_data=deriv_th_data,
                            uOpt_vel=uOpt_vel,
                            uOpt_angle=uOpt_angle,
                            x_coord=x_coord,
                            y_coord=y_coord,
                            th_coord=th_coord,
                            tau2=tau2
                           )
    return matlab_var_dict


#---------------------- Load MATLAB ---------------------------------------------------------
#v3 added uopt lookup table
matlab_var_dict_coarse= get_matlab_variables('/home/javier/jax_work/mppi/rc_car_mppi/brt_rc_wh_coarse_v3.mat')
data_coarse = matlab_var_dict_coarse['value_func_data']
data_lx = matlab_var_dict_coarse['lx_data']

matlab_var_dict_fine= get_matlab_variables('/home/javier/jax_work/mppi/rc_car_mppi/brt_rc_wh_fine_v3.mat')
data_fine  = matlab_var_dict_fine['value_func_data']

In [8]:
from ipywidgets import interact


def plot_func(theta):
    
    # Print the state with 2 decimals
    print('[th]')
    print(np.around(theta, decimals=2))
    # Get the index of the closest v and th to the state_first_fltr
    th_idx = np.argmin(np.abs(matlab_var_dict_coarse['th_coord'] - theta))

    fig, ax1 = plt.subplots(1, 1, figsize=(6, 4))

    ####race track#####
    data1 = data_lx
    data1 = data1.transpose()
    CP1 = ax1.contour(matlab_var_dict_coarse['x_coord'], matlab_var_dict_coarse['y_coord'], data1, 0, colors='red', linewidths=1)

    data2 = data_coarse[:, :, th_idx]  # Slice of vx
    data2 = data2.transpose()
    CP2 = ax1.contour(matlab_var_dict_coarse['x_coord'], matlab_var_dict_coarse['y_coord'], data2, 0, colors='blue', linewidths=1)
    
    data3 = data_fine[:, :, th_idx]  # Slice of vx
    data3 = data3.transpose()
    CP3 = ax1.contour(matlab_var_dict_fine['x_coord'], matlab_var_dict_fine['y_coord'], data3, 0, colors='green', linewidths=1)
       
    plt.show()

# Use interact to create a slider for idx_to_plot
interact(plot_func, theta=(-np.pi, np.pi, np.pi/18))

interactive(children=(FloatSlider(value=0.0, description='theta', max=3.141592653589793, min=-3.14159265358979…

<function __main__.plot_func(theta)>

In [9]:
import jax
import jax.numpy as jnp
from jax.lib import xla_bridge

# List all available devices
devices = jax.devices()
print("Available devices:", devices)

# Create an array and print its device
x = jnp.array([1.0, 2.0, 3.0])
print("Array device:", x.device_buffer.device())

# Print the backend being used
print("Backend platform:", xla_bridge.get_backend().platform)

Available devices: [gpu(id=0)]
Array device: gpu:0
Backend platform: gpu
